In [1]:
import sys
sys.path.append("..")

import datetime as dt
from ipywidgets import interact
import itertools
from kneed import KneeLocator
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_ta as pta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import statsmodels.api as sm
import vectorbtpro as vbt

from lib import bitget_loader, utils

In [ ]:
symbol = 'SOLUSDT'
is_start = dt.date(2022,1,1)
is_end = dt.date(2023,5,30)

os_start = dt.date(2023,6,1)
os_end = dt.date(2023,12,31)

df = bitget_loader.load_klines_in_date_range(symbol, is_start, os_end).tz_convert(None) # convert to tz naive so I can loc with dates with pandas
df = df.drop_duplicates() # bitget has overlaps in their data

# Percentage Stop

In [4]:
is_df = df.loc[is_start:is_end].copy()

In [5]:
pf = vbt.Portfolio.from_random_signals(
    close=is_df['close'], open=is_df['open'], high=is_df['high'], low=is_df['low'],
    n=2000,
    sl_stop=vbt.Param(np.arange(0.0005, 0.01, 0.00025)),
    # tp_stop=vbt.Param(np.arange(0.0005, 0.01, 0.0005)),
    direction='LongOnly',
    td_stop=vbt.Param(np.arange(5, 61, 5)),
    time_delta_format=0,
    # delta_format='Target'
    upon_long_conflict='Ignore'
)

In [6]:
wr = pf.trades.win_rate.unstack()
wr.index = wr.index.round(4)

In [7]:
knees = pd.Series([KneeLocator(x=wr.index, y=wr[td_stop]).knee for td_stop in wr.columns], wr.columns)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
wr.plot(ax=ax[0])
pd.Series([wr.loc[v, i] for i, v in knees.items()], knees).plot(marker='.', linestyle='', label='Knees', legend=True, ax=ax[0], ylabel='Win Rate', xlabel='Stop Loss (%)', grid=True)
ax[0].legend(ncols=3)
knees.plot(ax=ax[1], grid=True, ylabel='Stop Loss Knee', xlabel='Time Exit')
fig.tight_layout()

# ATR Stop

In [151]:
is_df['tr'] = pta.true_range(is_df['high'], is_df['low'], is_df['close'])
is_df['atr'] = is_df['tr'].ewm(60).mean()*np.sqrt(30)
is_df['natr'] = is_df['atr']/is_df['close']

In [152]:
atr_mults = np.arange(0.1, 2, 0.1)
td_stops = np.arange(10, 120, 10)

In [164]:
pf = vbt.Portfolio.from_random_signals(
    close=is_df['close'], open=is_df['open'], high=is_df['high'], low=is_df['low'],
    n=1000,
    sl_stop=vbt.Param([(is_df['natr']*x).rename(x) for x in atr_mults]),
    # tp_stop=vbt.Param(np.arange(0.0005, 0.01, 0.0005)),
    direction='ShortOnly',
    td_stop=vbt.Param(td_stops),
    time_delta_format=0,
    # delta_format='Target'
    upon_long_conflict='Ignore'
)

In [165]:
wr = pf.trades.win_rate.unstack()
wr.index = wr.index.str[7:].astype(int)
wr = wr.sort_index()
wr.index = atr_mults

In [166]:
knees = pd.Series([KneeLocator(x=wr.index, y=wr[td_stop]).knee for td_stop in wr.columns], wr.columns)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
wr.plot(ax=ax[0])
pd.Series([wr.loc[v, i] for i, v in knees.items()], knees).plot(marker='.', linestyle='', label='Knees', legend=True, ax=ax[0], ylabel='Win Rate', xlabel='Stop Loss (N ATRs)', grid=True)
ax[0].legend(ncols=3)
knees.plot(ax=ax[1], grid=True, ylabel='Stop Loss Knee', xlabel='Time Exit')
fig.tight_layout()

In [179]:
wrs = []
for barrier in [(is_df['natr']*x).rename(round(x, 2)) for x in atr_mults]:
    pf = vbt.Portfolio.from_random_signals(
    close=is_df['close'], open=is_df['open'], high=is_df['high'], low=is_df['low'],
    n=1000,
    sl_stop=barrier,
    tp_stop=barrier,
    direction='LongOnly',
    td_stop=vbt.Param(td_stops),
    time_delta_format=0,
    # delta_format='Target'
    upon_long_conflict='Ignore'
    )
    records = pf.trades.records
    records['td_stop'] = pf.trades.win_rate.index[records['col']]
    records['is_td_stop'] = (records['exit_idx'] - records['entry_idx']) == records['td_stop']
    wrs.append(records.groupby('col').apply(lambda d: d['is_td_stop'].mean()))

In [180]:
wr = pd.concat(wrs, axis=1)
wr.columns = atr_mults.round(2)
wr.index = td_stops

In [181]:
wr.columns.name = 'atr_mult'

In [ ]:
wr.plot(xlabel='td_stop', ylabel='Win Rate')
plt.axhline(0.33)
plt.grid()
plt.legend(ncols=4)

In [13]:
# records = pf.trades.records
# records['dt'] = is_df.index[records['entry_idx']]
# records['exit_dt'] = is_df.index[records['exit_idx']]
# records['sl'] = is_df['sl'].iloc[records['entry_idx']].values
# records['tp'] = is_df['tp'].iloc[records['entry_idx']].values
# records = records.set_index('dt')

In [14]:
# fig = go.FigureWidget(make_subplots(rows=1, cols=1, shared_xaxes=True))
# fig.add_trace(go.Candlestick(), row=1, col=1)
# fig.add_trace(go.Scatter(mode='markers'), row=1, col=1)
# fig.add_trace(go.Scatter(mode='markers'), row=1, col=1)
# fig.add_trace(go.Scatter(mode='markers', marker=dict(symbol='x', size=12, color='green')), row=1, col=1)
# fig.add_trace(go.Scatter(mode='markers', marker=dict(symbol='x', size=12, color='red')), row=1, col=1)
# fig.update_layout(height=800, margin=dict(l=20,r=20,b=20,t=20), xaxis=dict(rangeslider=dict(visible=False)))

# @interact(r=records['id'], col=df.columns, col2=df.columns)
# def update(r, col, col2):
#    with fig.batch_update():
#       _r = records[records['id']==r].iloc[0]
#       _sdf = is_df.iloc[_r['entry_idx']-20:_r['exit_idx']+20]
      
#       print(_r)
#       fig.data[0].x, fig.data[0].open, fig.data[0].high = _sdf.index, _sdf['open'], _sdf['high']
#       fig.data[0].low, fig.data[0].close = _sdf['low'], _sdf['close']
#       fig.data[1].x, fig.data[1].y = _sdf.index[20:-20], np.repeat(_r['sl'], _sdf.iloc[20:-20].shape[0])
#       fig.data[2].x, fig.data[2].y = _sdf.index[20:-20], np.repeat(_r['tp'], _sdf.iloc[20:-20].shape[0])
#       fig.data[3].x, fig.data[3].y = [_sdf.index[20]], [_r['entry_price']]
#       fig.data[4].x, fig.data[4].y = [_sdf.index[-20]], [_r['exit_price']]
#       fig.update_layout()
# fig